In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import ast
import requests
from bs4 import BeautifulSoup

In [4]:
# test description
nodeurl1 = "https://thelede.in/"
# making requests instance
reqs = requests.get(nodeurl1, timeout = 20)
# using the BeaitifulSoup module
soup = BeautifulSoup(reqs.text, 'html.parser')
for title in soup.find_all("meta", attrs={"name": "description"}):
    print(title.get("content"))


In [2]:
data = pd.read_csv("../lite_data/dec15_extracted_domains.csv", header=0)
data.head()

,Unnamed: 0,author,domain,count
0,0,NaN,1950.today,1
1,1,NaN,4lakhdenahoga.com,2
2,2,NaN,aajtak.in,32
3,3,NaN,abc.net.au,1
4,4,NaN,abplive.com,20


In [3]:
data2 = pd.read_csv("../lite_data/aap_dec15_extracted_domains.csv", header=0)
data2.head()

,Unnamed: 0,author,domain,count
0,0,NaN,1.si,1
1,1,NaN,54.xn--h2brj9c,1
2,2,NaN,5dariyanews.com,1
3,3,NaN,aajtak.in,8
4,4,NaN,aamaadmiparty.org,1


In [4]:
data = pd.concat([data, data2])

In [5]:
data = data.dropna(subset=['author'])

In [6]:
#map author and party handle
import glob
from collections import Counter, defaultdict

datafiles = glob.glob("/home/phadke/ONR/ONR/big_data/Twitter/*.csv")
author2party = defaultdict()

for d in datafiles:
    splitfilename = d.replace(".csv", "").split("_")
    party = splitfilename[len(splitfilename)-1]
    author = splitfilename[len(splitfilename)-2]
    author2party[author] = party


In [7]:
data['party'] = data['author'].apply(lambda x: author2party[x])

In [8]:
domlabs = pd.read_csv("../lite_data/Website_List.csv", header=0)
domlabs.head()

,UniqueID,Website_Name,Political_Leaning,Website_URL,Language,MobileSite,PrintMedia,TVMedia,OnlineMedia,FacebookReach,TwitterReach,InstagramReach,AlexaGlobalRank,AlexaIndiaRank,Parent_Company,Website_Registrar,Registered_On,Expires_On,TrafficContributingSubDomains
0,1.0,Aaj Tak,RIGHT_TO_RIGHTCENTRE,aajtak.intoday.in,Hindi,m.aajtak.in,N,Y,Y,26.2M,9.8M,3.7M,607.7K,98.2K,Living Media Group (TV Today Network Limited),MPS Infotecnics Limited,16/02/2005,16/02/2029,"{'m.aajtak.in': '93.83%', 'OTHER': '6.17%'}"
1,2.0,Dainik Jagran,RIGHT_TO_RIGHTCENTRE,jagran.com,Hindi,m.jagran.com,Y,N,Y,14.8M,1.1M,58.5K,1.7K,132,Jagran Prakashan Limited,"Network Solutions, LLC",17/01/1997,18/01/2030,"{'jagran.com': '70.51%', 'epaper.jagran.com': ..."
2,3.0,RVCJ News,NaN,rvcj.com,English,N,N,N,Y,14.5M,193.0K,2.5M,26.9K,3.8K,RVCJ Digital Media Private Limited,"NameCheap, Inc.",20/04/2007,20/04/2022,"{'rvcj.com': '99.04%', 'OTHER': '0.97%'}"
3,4.0,Dainik Bhaskar,RIGHT_TO_RIGHTCENTRE,bhaskar.com,Hindi,N,Y,N,Y,14.3M,1.0M,988.2K,1.3K,107,D B Corp Limited,PDR Ltd. d/b/a PublicDomainRegistry.com,17/04/1998,16/04/2029,"{'bhaskar.com': '72.22%', 'epaper.bhaskar.com'..."
4,5.0,Zee News,RIGHT_TO_RIGHTCENTRE,zeenews.india.com,English,N,N,Y,Y,11.4M,3.2M,1.8M,74.9K,7.6K,Essel Group,"GoDaddy.com, LLC",13/08/1997,30/05/2027,"{'cdn.zeenews.com': '53.01%', 'my.zeenews.com'..."


In [9]:
domlabs = domlabs.dropna(subset=['Political_Leaning'])
dom2label = defaultdict()

for idx, row in domlabs.iterrows():
    leaning = row['Political_Leaning']
    main_url = row['Website_URL']
    dom2label[main_url] = leaning
    traffic = ast.literal_eval(row['TrafficContributingSubDomains'])
    if traffic:
        for k in traffic.keys():
            if k!= 'OTHER':
                dom2label[k] = leaning

In [10]:
domlabel = data[['author','domain','party']].drop_duplicates()
domlabel.head()

,author,domain,party
411,01Bidar,drivespark.com,INC
412,01Bidar,indiatoday.in,INC
413,01Bidar,livehindustan.com,INC
414,01Bidar,prajavani.net,INC
415,01Bidar,shrts.in,INC


In [11]:
domain_by_party = domlabel.groupby(['domain','party']).size().reset_index()
domain_by_party.head()

,domain,party,0
0,00.pm,BJP,1
1,06-jun-20.click,INC,1
2,1.am,INC,1
3,1.as,BJP,2
4,1.book,BJP,11


In [12]:
domain_list = data.groupby("author")['domain'].apply(list).reset_index()
domain_list.head()

,author,domain
0,01Bidar,"[drivespark.com, indiatoday.in, livehindustan...."
1,02,"[abplive.com, expert.so, forms.gle, indiatoday..."
2,0403,"[abplive.com, amarujala.com, amitshah.co.in, b..."
3,07,[pscp.tv]
4,07Sharma4u,"[abplive.com, amazon.in, champions.gov.in, pmi..."


In [13]:
domain_list['domainsize'] = domain_list['domain'].apply(lambda x: len(set(x)))
print(len(domain_list))
print(len(domain_list.loc[domain_list['domainsize']>5]))

5085
3083


In [14]:
domain_list.head()

,author,domain,domainsize
0,01Bidar,"[drivespark.com, indiatoday.in, livehindustan....",6
1,02,"[abplive.com, expert.so, forms.gle, indiatoday...",6
2,0403,"[abplive.com, amarujala.com, amitshah.co.in, b...",22
3,07,[pscp.tv],1
4,07Sharma4u,"[abplive.com, amazon.in, champions.gov.in, pmi...",5


In [15]:
multiple_shared_authors = domain_list.loc[domain_list['domainsize']>5]
multiple_shared_authors.head()

,author,domain,domainsize
0,01Bidar,"[drivespark.com, indiatoday.in, livehindustan....",6
1,02,"[abplive.com, expert.so, forms.gle, indiatoday...",6
2,0403,"[abplive.com, amarujala.com, amitshah.co.in, b...",22
5,0914,"[abplive.com, abpnadu.com, archive.is, breezys...",83
6,1,"[abvp.org, app.link, bhaskar.com, bhaskarhindi...",15


In [16]:
nodes = defaultdict()
edges = defaultdict()

for idx, row in multiple_shared_authors.iterrows():
    authorlist = list(set(row['domain']))
    for a in authorlist:
        if a not in nodes:
            nodes[a] = 1
        else:
            nodes[a]+=1
            
    for i in range(len(authorlist)-1):
        for j in range(i+1, len(authorlist)):
            edgetuple = sorted([authorlist[i], authorlist[j]])
            if (edgetuple[0], edgetuple[1]) not in edges:
                edges[(edgetuple[0], edgetuple[1])] = 1
            else:
                edges[(edgetuple[0], edgetuple[1])] +=1


In [17]:
import networkx as nx

G = nx.Graph()

# for n in nodes.keys():
#     G.add_node(n, size=nodes[n], value=domainvalues[n])

ndlist = []
    
for e in edges:
    if edges[e] > 3:
        if e[0] not in ndlist:
            try:
                G.add_node(e[0], size=nodes[e[0]], party=dom2label[e[0]])
            except:
                G.add_node(e[0], size=nodes[e[0]], party="unknown")
            ndlist.append(e[0])
        if e[1] not in ndlist:
            try:
                G.add_node(e[1], size=nodes[e[1]], party=dom2label[e[1]])
            except:
                G.add_node(e[1], size=nodes[e[1]], party='unknown')
            ndlist.append(e[1])

        
        G.add_edge(e[0], e[1], weight=edges[e])

In [18]:
nodes = list(G.nodes)

In [19]:
nodes[0]

'indiatoday.in'

In [20]:
domaintitles = pd.DataFrame(columns=['domain', 'title_text'])

In [21]:
for n in nodes:
    print(n)
    nodeurl1 = "https://" + n
    nodeurl2 = "http://" + n
    text = ""
    
    try:
        # making requests instance
        reqs = requests.get(nodeurl1, timeout = 10)
        # using the BeaitifulSoup module
        soup = BeautifulSoup(reqs.text, 'html.parser')
        for title in soup.find_all('title'):
            text = text + title.get_text()
        

    except:
        try:
            reqs = requests.get(nodeurl2, timeout = 10)
            # using the BeaitifulSoup module
            soup = BeautifulSoup(reqs.text, 'html.parser')
            for title in soup.find_all('title'):
                text = text + title.get_text()

            
        except:
            text = ""
        
        
    if len(text) > 1:
        domaintitles = domaintitles.append({'domain':n, 'title_text':text}, ignore_index=True)
    else:
        domaintitles = domaintitles.append({'domain':n, 'title_text':'not_found'}, ignore_index=True)

indiatoday.in
shrts.in
livehindustan.com
prajavani.net
thehindu.com
time8.in
abplive.com
forms.gle
news18.com
thedailystar.net
indiatimes.com
hindustantimes.com
cybercrime.gov.in
pib.gov.in
bitly.ws
cowin.gov.in
narendramodi.in
padmaawards.gov.in
amarujala.com
bjp.org
amitshah.co.in
mygov.in
bjplive.org
washingtonpost.com
linkedin.com
sonarbangla2021.com
pscp.tv
indianrail.gov.in
nm-4.com
foreignpolicy.com
livemint.com
business-standard.com
reuters.com
timesofindia.com
india.com
ndtv.com
livelaw.in
timesnownews.com
chng.it
nytimes.com
outlookindia.com
sundayguardianlive.com
thecommunemag.com
wp.me
opindia.com
wikipedia.org
firstpost.com
tv9hindi.com
dharmadispatch.in
ecoti.in
tfipost.com
wionews.com
tinyurl.com
mathrubhumi.com
indiafacts.org
hindupost.in
dinamalar.com
indianarrative.com
dlvr.it
fllwrs.com
esamskriti.com
businessinsider.com
nakkheeran.in
rt.com
cutt.ly
tfiglobalnews.com
maalaimalar.com
nypost.com
abpnadu.com
f24.my
theconversation.com
bhaskarhindi.com
kooapp.com
patrika

scam.it
iyc.in
medium.com
newsnetworks.co.in
hon.pm
dnai.in
india.to
education.to
hackathons.to
khaskhabar.com
tiny.cc
theaustraliatoday.com.au
earshot.in
polimernews.com
ns7.tv
thanthitv.com
desiyamurasu.com
gn24.ae
delhishelterboard.in
gallantryawards.gov.in
oxygenhry.in
project-syndicate.org
vikatan.com
b.sc
werejectadmk.com
kalaignarseithigal.com
bloombergquint.com
counterview.net
epw.in
worldbank.org
newsdanka.com
covidfacts.in
vibesofindia.com
streamnow.in
gstv.in
incgujarat.com
timesgroup.com
sandeshnews.tv
joshhosh.com
virtualdeepotsav.com
belurmath.org
openthemagazine.com
jharkhandsandesh.in
khabarchhe.com
thecrediblehistory.com
ft.com
gov.in
shar.es
news9live.com
theleaflet.in
soundcloud.com
independent.co.uk
saamtv.com
mr.pm
newsnetwork24x7.com
agnialok.com
modipara.com
keralakaumudi.com
windy.com
thefameindia.com
assamtribune.com
newslivetv.com
asiatimes.com
nsuiquiz.com
minnambalam.com
hastakshep.com
banglahunt.com
sadhguru.org
punchnamu.com
janamtv.com
deccanchronicle.com

In [22]:
domaintitles.to_csv("./domaintitles.csv")

In [23]:
domaintitles.head()

,domain,title_text
0,indiatoday.in,"Latest News, Breaking News Today - Bollywood, ..."
1,shrts.in,Invalid Dynamic Link
2,livehindustan.com,Access Denied
3,prajavani.net,"Prajavani | ಪ್ರಜಾವಾಣಿ ತಾಜಾ ಸುದ್ದಿ, ಸಮಾಚಾರ, ವಾರ..."
4,thehindu.com,"\nThe Hindu: Breaking News, India News, Sports..."


In [25]:
domaintitles.sample(5)

,domain,title_text
205,thewirehindi.com,The Wire - Hindi - Hindi News (हिंदी न्यूज़): L...
323,hwnews.in,Latest Political News | HW News English | 24×7...
243,fb.me,Facebook - Log In or Sign Up
664,mynation.com,"MyNation: Breaking News, India News, Bollywood..."
18,amarujala.com,Hindi News (हिन्दी न्यूज़): Latest News In Hin...


In [26]:
len(domaintitles)

1068

In [27]:
domaintitles.sample(5)

,domain,title_text
1040,thefauxy.com,"The Fauxy – India's Finest, Fastest & Fictitio..."
244,tv9kannada.com,"TV9 Kannada News: Kannada Latest News, ಟಿವಿ9 ಕ..."
31,business-standard.com,"Business News, Finance News, India News, BSE/N..."
133,jansatta.com,Access Denied
25,sonarbangla2021.com,not_found


In [28]:
domaintitles.tail(5)

,domain,title_text
1063,jan-sampark.nic.in,Jansampark
1064,covidggn.com,\r\n\tHome Page : GC3\r\n
1065,starofmysore.com,Star of Mysore Online
1066,samvada.org,Vishwa Samvada Kendra – Vishwa Samvada Kendra
1067,congressor5guarantee.in,Congressor 5 Guarantee | Register for Congress...


In [29]:
def is_news(text):
    if "news" in text.lower():
        return 1
    
domaintitles['is_news'] = domaintitles['title_text'].apply(lambda x: is_news(x))

In [30]:
domaintitles.loc[domaintitles['is_news']==1]

,domain,title_text,is_news
0,indiatoday.in,"Latest News, Breaking News Today - Bollywood, ...",1.0
3,prajavani.net,"Prajavani | ಪ್ರಜಾವಾಣಿ ತಾಜಾ ಸುದ್ದಿ, ಸಮಾಚಾರ, ವಾರ...",1.0
4,thehindu.com,"\nThe Hindu: Breaking News, India News, Sports...",1.0
6,abplive.com,"Hindi News, Breaking News in Hindi, हिंदी न्यू...",1.0
8,news18.com,"News, Breaking News, Latest News, News Headlin...",1.0
...,...,...,...
1052,foxbusiness.com,Fox Business | Business News &Stock Quotes - S...,1.0
1053,munaadi.com,Munaadi - Voice of Voiceless - Chhattisgarh Ne...,1.0
1055,magzter.com,Magzter - World’s largest digital newsstand wi...,1.0
1056,threesocieties.com,Home - Three Societies - News Magazine,1.0


In [32]:
domaintitles.loc[domaintitles['title_text']=='not_found']

,domain,title_text,is_news
11,hindustantimes.com,not_found,NaN
12,cybercrime.gov.in,not_found,NaN
17,padmaawards.gov.in,not_found,NaN
23,washingtonpost.com,not_found,NaN
25,sonarbangla2021.com,not_found,NaN
...,...,...,...
1031,you.you,not_found,NaN
1037,hon.cm,not_found,NaN
1041,tl.gd,not_found,NaN
1050,corruption.as,not_found,NaN


In [33]:
def retry_titlessearch(row):
    if row['title_text'] == 'not_found':
        
        nodeurl1 = "https://" + row['domain']
        nodeurl2 = "http://" + row['domain']
        text = ""

        try:
            # making requests instance
            reqs = requests.get(nodeurl1, timeout = 20)
            # using the BeaitifulSoup module
            soup = BeautifulSoup(reqs.text, 'html.parser')
            for title in soup.find_all('title'):
                text = text + title.get_text()


        except:
            try:
                reqs = requests.get(nodeurl2, timeout = 20)
                # using the BeaitifulSoup module
                soup = BeautifulSoup(reqs.text, 'html.parser')
                for title in soup.find_all('title'):
                    text = text + title.get_text()


            except:
                text = ""


        if len(text) > 1:
            return text
        else:
            return "not_found"
        
        
    else:
        return row['title_text']
       

In [35]:
domaintitles['title_text'] = domaintitles.apply(lambda row: retry_titlessearch(row), axis=1)

In [36]:
domaintitles['is_news'] = domaintitles['title_text'].apply(lambda x: is_news(x))

In [37]:
domaintitles.loc[domaintitles['is_news']==1]

,domain,title_text,is_news
0,indiatoday.in,"Latest News, Breaking News Today - Bollywood, ...",1.0
3,prajavani.net,"Prajavani | ಪ್ರಜಾವಾಣಿ ತಾಜಾ ಸುದ್ದಿ, ಸಮಾಚಾರ, ವಾರ...",1.0
4,thehindu.com,"\nThe Hindu: Breaking News, India News, Sports...",1.0
6,abplive.com,"Hindi News, Breaking News in Hindi, हिंदी न्यू...",1.0
8,news18.com,"News, Breaking News, Latest News, News Headlin...",1.0
...,...,...,...
1052,foxbusiness.com,Fox Business | Business News &Stock Quotes - S...,1.0
1053,munaadi.com,Munaadi - Voice of Voiceless - Chhattisgarh Ne...,1.0
1055,magzter.com,Magzter - World’s largest digital newsstand wi...,1.0
1056,threesocieties.com,Home - Three Societies - News Magazine,1.0


In [38]:
domaintitles.loc[(domaintitles['is_news']!=1)&(domaintitles['title_text']!='not_found')]

,domain,title_text,is_news
1,shrts.in,Invalid Dynamic Link,NaN
2,livehindustan.com,Access Denied,NaN
5,time8.in,Please Wait... | Cloudflare,NaN
7,forms.gle,Invalid Dynamic Link,NaN
13,pib.gov.in,\r\n\tPress Information Bureau\r\n,NaN
...,...,...,...
1063,jan-sampark.nic.in,Jansampark,NaN
1064,covidggn.com,\r\n\tHome Page : GC3\r\n,NaN
1065,starofmysore.com,Star of Mysore Online,NaN
1066,samvada.org,Vishwa Samvada Kendra – Vishwa Samvada Kendra,NaN


In [39]:
domaintitles.to_csv("./domaintitles.csv")